# Models

> Basic Model redirections happen here. You define a model, and call a get_model function from this notebook. After that, the notebook redirectes the model to the correct model type.


In [ ]:
#| default_exp models

In [ ]:
#| hide
from nbdev.showdoc import *  # noqa: F403

In [ ]:
#| export
from fastcore.utils import *  # noqa: F403
from transformers import AutoModelForCausalLM
import torch
from torch import nn
from peft import *  # noqa: F403
from fedai.vision.models import *  # noqa: F403
from fedai.text.models import *  # noqa: F403

In [ ]:
#| export 
class LogisticRegression(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.linear(x)

In [ ]:
# export
class MLP(nn.Module):
    def __init__(self, dim_in, dim_hidden, dim_out):
        super(MLP, self).__init__()
        self.layer_input = nn.Linear(dim_in, dim_hidden)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout()
        self.layer_hidden = nn.Linear(dim_hidden, dim_out)

    def forward(self, x):
        x = x.view(-1, x.shape[1]*x.shape[-2]*x.shape[-1])
        x = self.layer_input(x)
        x = self.dropout(x)
        x = self.relu(x)
        x = self.layer_hidden(x)
        return x

In [ ]:
#| export

def get_model(cfg):
    model_name = cfg.model.name

    # Check if the model name contains "hf://"
    if model_name.startswith("hf://"):
        return get_hf_model(cfg)  # type: ignore # Call your HF model loader function  # noqa: F405

    # Define the rest of the model mapping
    mapping = {
        "MNISTCNN": MNISTCNN(num_classes=10),  # noqa: F405 # type: ignore
        "CIFAR10CNN": CIFAR10CNN(num_classes=10),  # noqa: F405 # type: ignore
        
        "MLP": MLP(
            dim_in=cfg.model.dim_in, 
            dim_hidden=cfg.model.dim_hidden, 
            dim_out=cfg.model.dim_out
            ),

        "CharacterLSTM": CharacterLSTM(  # noqa: F405 # type: ignore
            vocab_size=cfg.model.vocab_size,
            embed_size=cfg.model.embed_size,
            hidden_size=cfg.model.hidden_size,
            num_layers=cfg.model.num_layers
        )
    }

    # Look up the model in the mapping
    if model_name in mapping:
        return mapping[model_name]
    
    raise ValueError(f"Model '{model_name}' is not recognized, the available models are: {list(mapping.keys())}")


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()  # noqa: E702